In [1]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils
import spacy
from collections import defaultdict
from nltk.corpus import wordnet as wn

In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [4]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [5]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))
dataset = rel_ext.Dataset(corpus, kb)

In [6]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.05, 0.75, 0.20],
    seed=1)

splits

{'tiny': Corpus with 14,966 examples; KB with 2,201 triples,
 'train': Corpus with 251,793 examples; KB with 34,435 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

In [7]:
def directional_bag_of_non_stop_words_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        words = remove_stop_words(ex.middle)
        for word in words:
            feature_counter[word + subject_object_suffix] += 1 
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        words = remove_stop_words(ex.middle)
        for word in words:
            feature_counter[word + object_subject_suffix] += 1

    return feature_counter

def directional_bag_of_words_right_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        words = remove_stop_words(ex.right)
        for word in words:
            feature_counter[word + subject_object_suffix] += 1  
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        words = remove_stop_words(ex.right)
        for word in words:
            feature_counter[word + subject_object_suffix] += 1 

    return feature_counter


def remove_stop_words(sentence):
    words = []
    for word in sentence.split(' '):
        if word.lower() not in spacy_stopwords:
            words.append(word)
            
    return words

In [8]:
def entity_mentions_featurizer(kbt, corpus, feature_counter):

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        words = ex.left.split(' ') + ex.right.split(' ')
        for word in words:
            if word == kbt.sbj or word == kbt.obj:
                feature_counter[word] += 1          
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        words = ex.left.split(' ') + ex.right.split(' ')
        for word in words:
            if word == kbt.sbj or word == kbt.obj:
                feature_counter[word] += 1          
            
    return feature_counter

In [9]:
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        words = ex.middle.split(' ')
        for word in words:
            feature_counter[word + subject_object_suffix] += 1          
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        words = ex.middle.split(' ')
        for word in words:
            feature_counter[word + object_subject_suffix] += 1
            
    return feature_counter

def directional_bag_of_bigrams_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        words = ex.middle.split(' ')
        for word in get_word_bigrams(words):            
            feature_counter[word + subject_object_suffix] += 1
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        words = ex.middle.split(' ')
        for word in get_word_bigrams(words):            
            feature_counter[word + subject_object_suffix] += 1
            
    return feature_counter

def directional_bag_of_trigrams_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        words = ex.middle.split(' ')
        for word in get_word_trigrams(words):            
            feature_counter[word + subject_object_suffix] += 1            
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        words = ex.middle.split(' ')
        for word in get_word_trigrams(words):            
            feature_counter[word + subject_object_suffix] += 1
            
    return feature_counter

def get_word_bigrams(words):
    result = []
    if len(words) > 0:
        for index in range(len(words) - 1):
            result.append(words[index] + ' ' + words[index + 1])
    
    return result

def get_word_trigrams(words):
    result = []
    if len(words) > 0:
        for index in range(len(words) - 2):
            result.append(words[index] + ' ' + words[index + 1] + ' ' + words[index + 2])
    
    return result

In [10]:
def directional_bag_of_words_right_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.right.split(' '):            
            feature_counter[word + subject_object_suffix] += 1 
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.right.split(' '):
            feature_counter[word + object_subject_suffix] += 1

    return feature_counter

In [11]:
def directional_bag_of_words_left_featurizer(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.left.split(' '):            
            feature_counter[word + subject_object_suffix] += 1 
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.left.split(' '):
            feature_counter[word + object_subject_suffix] += 1

    return feature_counter

In [12]:
def middle_length_featurizer(kbt, corpus, feature_counter):
    
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        feature_counter[str(len(ex.middle))] += 1 
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        feature_counter[str(len(ex.middle))] += 1

    return feature_counter

In [13]:
def left_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        tags = get_tags(ex.left_POS)
        for tag in get_tag_bigrams(tags):
            feature_counter[tag] += 1
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        tags = get_tags(ex.left_POS)
        for tag in get_tag_bigrams(tags):
            feature_counter[tag] += 1

    return feature_counter

def right_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        tags = get_tags(ex.right_POS)
        for tag in get_tag_bigrams(tags):
            feature_counter[tag] += 1
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        tags = get_tags(ex.right_POS)
        for tag in get_tag_bigrams(tags):
            feature_counter[tag] += 1

    return feature_counter

def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        tags = get_tags(ex.middle_POS)
        for tag in get_tag_bigrams(tags):
            feature_counter[tag] += 1
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        tags = get_tags(ex.middle_POS)
        for tag in get_tag_bigrams(tags):
            feature_counter[tag] += 1

    return feature_counter


def get_tag_bigrams(tags):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"

    ##### YOUR CODE HERE
    result = []
    if len(tags) > 0:
        result = [start_symbol + ' ' + tags[0]]
        for index in range(len(tags) - 1):
            result.append(tags[index] + ' ' + tags[index + 1])
 
        result.append(tags[len(tags) - 1] + ' ' + end_symbol)
    
    return result 


def get_tags(s):
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)

In [14]:
def synset_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        synsets = get_synsets(ex.middle_POS)
        for s in synsets:
            feature_counter[s] += 1
    
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        synsets = get_synsets(ex.middle_POS)
        for s in synsets:
            feature_counter[s] += 1

    return feature_counter


def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets
    associated with elements of `s`.
    """
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]

    synsets = []
    ##### YOUR CODE HERE
    for w, t in wt:
        wnt = convert_tag(t)
        wns = wn.synsets(w, wnt)
        for s in wns:
            synsets.append(str(s))

    return synsets


def convert_tag(t):
    """Converts tags so that they can be used by WordNet:

    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'j':
        return 'a'
    else:
        return None

In [15]:
kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
feature_counter = defaultdict(int)
test = entity_mentions_featurizer(kbt, corpus, feature_counter)
test

defaultdict(int, {'xkcd': 2})

#### Experiment 1 - LogisticRegression

*Note - removing stop words decreased the precision and recall.*

*Note - adding features derived from words left to the object/subject and right to them, decreased the precision and recall too.*

*Note - bigrams and trigrams based on the words between the two entity mentions decreased the performance of the model.*

##### 1.a 

LogisticRegression model (solver='liblinear')

with large 0.75 of total

directional_bag_of_words_featurizer

macro-average             0.764      0.356      0.605

with tiny 0.05 of total

directional_bag_of_words_featurizer

macro-average             0.714      0.230      0.440 (with stop words)

directional_bag_of_words_featurizer + middle_length_featurizer

macro-average             0.696      0.216      0.450


with tiny 0.15 of total

directional_bag_of_words_featurizer,
                                middle_bigram_pos_tag_featurizer,
                                synset_featurizer,
                                middle_length_featurizer

macro-average             0.730      0.354      0.579


with train 0.75 of total

middle_bigram_pos_tag_featurizer + directional_bag_of_words_featurizer + synset_featurizer

macro-average             0.738      0.417      0.627

with tiny 0.05 of total

directional_bag_of_words + middle_bigram_pos_tag_featurizer + synset_featurizer

macro-average             0.702      0.266      0.503

directional_bag_of_words + middle_bigram_pos_tag_featurizer

macro-average             0.764      0.243      0.497

directional_bag_of_words + directional_bigrams_of_words

macro-average             0.747      0.230      0.474

directional_bag_of_words + bigrams_of_words

macro-average             0.795      0.218      0.465

bigrams_of_words_featurizer

macro-average             0.794      0.173      0.396

In [16]:
model_factory = lambda: LogisticRegression(
    fit_intercept=True, solver='liblinear', max_iter=300)

In [17]:
lr_results = rel_ext.experiment(splits, train_split='train', test_split='dev', 
                                featurizers=[directional_bag_of_words_featurizer,
                                             middle_bigram_pos_tag_featurizer,
                                             synset_featurizer,
                                             middle_length_featurizer], 
                                model_factory=model_factory,
                                verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.840      0.403      0.691        340       5716
author                    0.833      0.656      0.790        509       5885
capital                   0.633      0.200      0.442         95       5471
contains                  0.769      0.772      0.770       3904       9280
film_performance          0.818      0.706      0.793        766       6142
founders                  0.709      0.442      0.633        380       5756
genre                     0.701      0.400      0.609        170       5546
has_sibling               0.877      0.273      0.608        499       5875
has_spouse                0.905      0.367      0.700        594       5970
is_a                      0.682      0.350      0.574        497       5873
nationality               0.579      0.279      0.477        301       5677
parents     

In [18]:
rel_ext.examine_model_weights(lr_results)

Highest and lowest feature weights for relation adjoins:

     1.593 into_SO
     1.539 Córdoba_OS
     1.489 Valais_OS
     ..... .....
    -1.099 46
    -1.113 for_SO
    -1.200 <s> NN

Highest and lowest feature weights for relation author:

     2.715 by_SO
     1.820 Dossie_SO
     1.808 Easton_SO
     ..... .....
    -1.662 <s> TO
    -2.060 's_SO
    -2.551 by_OS

Highest and lowest feature weights for relation capital:

     1.186 including_SO
     1.112 Synset('airport.n.01')
     1.067 Tortola_OS
     ..... .....
    -0.992 . </s>
    -1.197 CC </s>
    -1.495 Glamorgan_OS

Highest and lowest feature weights for relation contains:

     2.648 third-largest_SO
     2.404 Ceuta_SO
     2.187 after_SO
     ..... .....
    -2.800 peninsula_OS
    -5.553 Bronx_OS
    -5.553 Synset('bronx.n.01')

Highest and lowest feature weights for relation film_performance:

     2.740 with_OS
     2.552 Synset('movie.n.01')
     2.549 alongside_OS
     ..... .....
    -1.922 film_OS
    -1.980

##### Experiment 2 SVC with RBF 

with train 0.75 of total

middle_bigram_pos_tag_featurizer + directional_bag_of_words_featurizer + synset_featurizer

macro-average             0.908      0.264      0.594

directional_bag_of_words_featurizer

macro-average             0.815      0.274      0.573

with tiny 0.05 of total

middle_bigram_pos_tag_featurizer + directional_bag_of_words_featurizer + synset_featurizer

macro-average             0.910      0.223      0.542

directional_bag_of_words_featurizer

macro-average             0.624      0.234      0.457


linear with tiny 0.15 of total

middle_bigram_pos_tag_featurizer,
                               directional_bag_of_words_featurizer,
                               synset_featurizer,
                               middle_length_featurizer

macro-average             0.577      0.463      0.536

RBF with tiny 0.15 of total

middle_bigram_pos_tag_featurizer,
                               directional_bag_of_words_featurizer,
                               synset_featurizer,
                               middle_length_featurizer

macro-average             0.911      0.231      0.560


In [19]:
from sklearn.svm import SVC

In [20]:
svcl_model_factory = lambda: SVC(kernel='linear')

In [21]:
svcl_results = rel_ext.experiment(splits, train_split='train', test_split='dev', 
                                  featurizers=[middle_bigram_pos_tag_featurizer,
                                               directional_bag_of_words_featurizer,
                                               synset_featurizer,
                                               middle_length_featurizer], 
                                  model_factory=svcl_model_factory,
                                  verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.692      0.403      0.605        340       5716
author                    0.730      0.680      0.719        509       5885
capital                   0.491      0.274      0.423         95       5471
contains                  0.754      0.833      0.769       3904       9280
film_performance          0.740      0.709      0.733        766       6142
founders                  0.583      0.497      0.564        380       5756
genre                     0.573      0.506      0.558        170       5546
has_sibling               0.720      0.263      0.534        499       5875
has_spouse                0.753      0.365      0.621        594       5970
is_a                      0.535      0.380      0.495        497       5873
nationality               0.433      0.352      0.414        301       5677
parents     

In [22]:
rel_ext.examine_model_weights(svcl_results)

Highest and lowest feature weights for relation adjoins:

     1.435 Valais_OS
     1.409 Córdoba_OS
     1.382 Synset('nearby.s.01')
     ..... .....
    -0.778 , VBZ
    -0.895 Baffin_SO
    -1.000 Tyrone_SO

Highest and lowest feature weights for relation author:

     1.390 1920_OS
     1.342 by_SO
     1.227 '_OS
     ..... .....
    -1.334 1985_OS
    -1.447 's_SO
    -1.776 by_OS

Highest and lowest feature weights for relation capital:

     1.000 Lenzie_OS
     1.000 Golmud_OS
     0.981 burgraviates_SO
     ..... .....
    -0.673 NN -RRB-
    -0.711 Patna_SO
    -0.969 Glamorgan_OS

Highest and lowest feature weights for relation contains:

     2.963 third-largest_SO
     2.488 Ceuta_SO
     2.022 Gozo_SO
     ..... .....
    -2.413 peninsula_OS
    -3.842 Synset('bronx.n.01')
    -3.842 Bronx_OS

Highest and lowest feature weights for relation film_performance:

     1.802 with_OS
     1.674 alongside_OS
     1.423 co-star_OS
     ..... .....
    -1.350 CC VBG
    -1.474 st

In [23]:
svcr_model_factory = lambda: SVC(gamma=2, C=1)

In [24]:
svcr_results = rel_ext.experiment(splits, train_split='train', test_split='dev', 
                                  featurizers=[middle_bigram_pos_tag_featurizer,
                                               directional_bag_of_words_featurizer,
                                               synset_featurizer,
                                               middle_length_featurizer], 
                                  model_factory=svcr_model_factory,
                                  verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   1.000      0.171      0.507        340       5716
author                    0.903      0.346      0.683        509       5885
capital                   0.783      0.189      0.481         95       5471
contains                  0.757      0.346      0.612       3904       9280
film_performance          0.816      0.168      0.461        766       6142
founders                  0.859      0.208      0.528        380       5756
genre                     0.737      0.165      0.435        170       5546
has_sibling               0.921      0.305      0.656        499       5875
has_spouse                0.932      0.345      0.695        594       5970
is_a                      0.898      0.266      0.608        497       5873
nationality               0.921      0.462      0.768        301       5677
parents     

##### Experiment 3

k-nearest (default parameters) and directional_bag_of_words_featurizer

macro-average             0.352      0.437      0.338

In [25]:
from sklearn.neighbors import KNeighborsClassifier

In [26]:
knc_model_factory = lambda: KNeighborsClassifier()

In [27]:
knc_results = rel_ext.experiment(splits, train_split='train', test_split='dev', 
                                 featurizers=[middle_bigram_pos_tag_featurizer,
                                              directional_bag_of_words_featurizer,
                                              synset_featurizer,
                                              middle_length_featurizer], 
                                 model_factory=knc_model_factory,
                                 verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.455      0.526      0.468        340       5716
author                    0.563      0.764      0.594        509       5885
capital                   0.240      0.463      0.266         95       5471
contains                  0.741      0.741      0.741       3904       9280
film_performance          0.601      0.658      0.611        766       6142
founders                  0.416      0.555      0.438        380       5756
genre                     0.404      0.482      0.418        170       5546
has_sibling               0.456      0.543      0.471        499       5875
has_spouse                0.521      0.613      0.537        594       5970
is_a                      0.402      0.555      0.426        497       5873
nationality               0.409      0.638      0.441        301       5677
parents     

##### Experiment 4 Multi-layer perception classifier

with train dataset 0.75 of total

directional_bag_of_words_featurizer with learning_rate_init=0.001

macro-average             0.785      0.396      0.637 (with alpha=0.17)


with tiny dataset 0.05 or total

middle_bigram_pos_tag_featurizer, directional_bag_of_words_featurizer, synset_featurizer

macro-average             0.702      0.459      0.620


directional_bag_of_words_featurizer with learning_rate_init=0.001

macro-average             0.649      0.364      0.522 (with alpha=0.17)

macro-average             0.649      0.355      0.517 (with alpha=0.18)

macro-average             0.658      0.339      0.517 (with alpha=0.2)

macro-average             0.644      0.357      0.516 (with alpha=0.15)

macro-average             0.578      0.419      0.511 (with alpha=0.1)

macro-average             0.703      0.273      0.502 (with alpha=0.3)

macro-average             0.528      0.451      0.495 (with alpha=0.05)

macro-average             0.742      0.249      0.490 (with alpha=0.5)


with learning_rate_init=0.1

directional_bag_of_words_featurizer

macro-average             0.615      0.366      0.500 (with alpha=0.2)

directional_bag_of_words_featurizer, middle_length_featurizer

macro-average             0.645      0.411      0.550 (with alpha=0.2)

directional_bag_of_words_featurizer, middle_length_featurizer, 
middle_bigram_pos_tag_featurizer

macro-average             0.656      0.417      0.566 (with alpha=0.2)

directional_bag_of_words_featurizer, middle_length_featurizer, middle_bigram_pos_tag_featurizer, synset_featurizer

macro-average             0.653      0.416      0.554 (with alpha=0.2)

with tiny dataset 0.15 or total and learning rate of 0.1

directional_bag_of_words_featurizer, middle_length_featurizer, middle_bigram_pos_tag_featurizer

macro-average             0.764      0.446      0.642 (with alpha=0.2)

with learning_rate_init=0.01

directional_bag_of_words_featurizer, middle_length_featurizer, 
middle_bigram_pos_tag_featurizer

macro-average             0.648      0.458      0.584 

directional_bag_of_words_featurizer 

macro-average             0.622      0.344      0.510 (with alpha=0.2)

In [28]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [29]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [30]:
def glove_middle_featurizer(kbt, corpus, np_func=np.sum):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)                        
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [31]:
from sklearn.neural_network import MLPClassifier

In [32]:
mlpc_model_factory = lambda: MLPClassifier(
    alpha=0.05, learning_rate_init=0.01, max_iter=500)

In [33]:
mlpc_glv_results = rel_ext.experiment(splits, train_split='train', test_split='dev', 
                                      featurizers=[glove_middle_featurizer], 
                                      model_factory=mlpc_model_factory,
                                      vectorize=False,
                                      verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.822      0.409      0.684        340       5716
author                    0.595      0.462      0.562        509       5885
capital                   0.378      0.295      0.358         95       5471
contains                  0.592      0.594      0.592       3904       9280
film_performance          0.528      0.415      0.501        766       6142
founders                  0.502      0.292      0.439        380       5756
genre                     0.291      0.094      0.205        170       5546
has_sibling               0.542      0.246      0.437        499       5875
has_spouse                0.542      0.263      0.447        594       5970
is_a                      0.345      0.245      0.319        497       5873
nationality               0.526      0.302      0.458        301       5677
parents     

In [35]:
mlpc_model_factory = lambda: MLPClassifier(
    alpha=0.1, learning_rate_init=0.01, max_iter=500)

In [36]:
mlpc_results = rel_ext.experiment(splits, train_split='train', test_split='dev', 
                                  featurizers=[directional_bag_of_words_featurizer,
                                               middle_length_featurizer,
                                               middle_bigram_pos_tag_featurizer], 
                                  model_factory=mlpc_model_factory,
                                  verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.906      0.594      0.820        340       5716
author                    0.835      0.825      0.833        509       5885
capital                   0.638      0.316      0.530         95       5471
contains                  0.779      0.858      0.794       3904       9280
film_performance          0.833      0.676      0.796        766       6142
founders                  0.740      0.650      0.720        380       5756
genre                     0.717      0.447      0.640        170       5546
has_sibling               0.785      0.521      0.713        499       5875
has_spouse                0.848      0.678      0.808        594       5970
is_a                      0.752      0.507      0.686        497       5873
nationality               0.756      0.668      0.736        301       5677
parents     

In [37]:
coefs = mlpc_results['models']['adjoins'].coefs_

In [38]:
sorted_weights = sorted([(wgt, idx) for idx, wgt in enumerate(coefs[1])], reverse=True)

In [39]:
from sklearn.linear_model import SGDClassifier

In [40]:
sgd_model_factory = lambda: SGDClassifier(alpha=0.0001)

In [41]:
sgd_results = rel_ext.experiment(splits, train_split='train', test_split='dev', 
                                  featurizers=[directional_bag_of_words_featurizer,
                                               middle_length_featurizer,
                                               middle_bigram_pos_tag_featurizer], 
                                  model_factory=sgd_model_factory,
                                  verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.823      0.438      0.700        340       5716
author                    0.817      0.666      0.781        509       5885
capital                   0.519      0.295      0.450         95       5471
contains                  0.777      0.768      0.775       3904       9280
film_performance          0.783      0.714      0.768        766       6142
founders                  0.699      0.439      0.625        380       5756
genre                     0.598      0.429      0.555        170       5546
has_sibling               0.746      0.265      0.547        499       5875
has_spouse                0.842      0.387      0.682        594       5970
is_a                      0.557      0.256      0.451        497       5873
nationality               0.516      0.272      0.438        301       5677
parents     

In [42]:
sgd_glv_results = rel_ext.experiment(splits, train_split='train', test_split='dev', 
                                  featurizers=[glove_middle_featurizer], 
                                     model_factory=sgd_model_factory,
                                     vectorize=False,
                                     verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.893      0.468      0.756        340       5716
author                    0.654      0.458      0.603        509       5885
capital                   0.299      0.242      0.285         95       5471
contains                  0.595      0.442      0.556       3904       9280
film_performance          0.728      0.325      0.583        766       6142
founders                  0.570      0.192      0.409        380       5756
genre                     0.429      0.088      0.242        170       5546
has_sibling               0.658      0.154      0.398        499       5875
has_spouse                0.715      0.364      0.599        594       5970
is_a                      0.562      0.119      0.322        497       5873
nationality               0.644      0.186      0.431        301       5677
parents     